In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Ruta del archivo GeoPackage
MAPA_TIPO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\Yucatán\Mapa_Tipologías_YUC.gpkg"

# Leer la capa GeoPackage
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias_Filtrado')

# Agrupar por tipología y calcular las estadísticas
resultados_tipologia = MAPA_TIPO_gdf.groupby('TIPOLOGIA').agg({
    'POPTOT2010': 'sum',
    'POPTOT2020': 'sum',
    'VIVTOT2010': 'sum',
    'VIVTOT2020': 'sum',
    'geometry': 'count',
    'area_ha': 'sum'
}).reset_index()

# Calcular el total
total = resultados_tipologia.sum(numeric_only=True)

# Agregar el total a los resultados
resultados_tipologia.loc['Total'] = total

# Calcular los porcentajes de cada tipología en el total
resultados_tipologia['% Población Total 2010'] = (resultados_tipologia['POPTOT2010'] / resultados_tipologia.loc['Total', 'POPTOT2010']) * 100
resultados_tipologia['% Población Total 2020'] = (resultados_tipologia['POPTOT2020'] / resultados_tipologia.loc['Total', 'POPTOT2020']) * 100
resultados_tipologia['% Viviendas Total 2010'] = (resultados_tipologia['VIVTOT2010'] / resultados_tipologia.loc['Total', 'VIVTOT2010']) * 100
resultados_tipologia['% Viviendas Total 2020'] = (resultados_tipologia['VIVTOT2020'] / resultados_tipologia.loc['Total', 'VIVTOT2020']) * 100
resultados_tipologia['% Manzanas'] = (resultados_tipologia['geometry'] / resultados_tipologia.loc['Total', 'geometry']) * 100
resultados_tipologia['% Area (ha)'] = (resultados_tipologia['area_ha'] / resultados_tipologia.loc['Total', 'area_ha']) * 100

# Calcular la diferencia de población y vivienda entre 2020 y 2010
resultados_tipologia['Diferencia Población'] = resultados_tipologia['POPTOT2020'] - resultados_tipologia['POPTOT2010']
resultados_tipologia['Diferencia Viviendas'] = resultados_tipologia['VIVTOT2020'] - resultados_tipologia['VIVTOT2010']

# Calcular la tasa de crecimiento por tipología y total
resultados_tipologia['Tasa Crecimiento Población (%)'] = (((resultados_tipologia['POPTOT2020'] / resultados_tipologia['POPTOT2010']) ** (1/10)) - 1) * 100
resultados_tipologia['Tasa Crecimiento Viviendas (%)'] = (((resultados_tipologia['VIVTOT2020'] / resultados_tipologia['VIVTOT2010']) ** (1/10)) - 1) * 100

# Calcular la cantidad de geometrías sobre área, población sobre área, y población sobre vivienda de 2010 y 2020
resultados_tipologia['Promedio de tamaño de Manzana'] = resultados_tipologia['geometry'] / resultados_tipologia['area_ha']
resultados_tipologia['Densidad poblacional 2010'] = resultados_tipologia['POPTOT2010'] / resultados_tipologia['area_ha']
resultados_tipologia['Densidad poblacional 2020'] = resultados_tipologia['POPTOT2020'] / resultados_tipologia['area_ha']
resultados_tipologia['Densidad habitacional 2010'] = resultados_tipologia['VIVTOT2010'] / resultados_tipologia['area_ha']
resultados_tipologia['Densidad habitacional 2020'] = resultados_tipologia['VIVTOT2020'] / resultados_tipologia['area_ha']
resultados_tipologia['Habitantes por vivienda 2010'] = resultados_tipologia['POPTOT2010'] / resultados_tipologia['VIVTOT2010']
resultados_tipologia['Habitantes por vivienda 2020'] = resultados_tipologia['POPTOT2020'] / resultados_tipologia['VIVTOT2020']

# Renombrar las columnas
resultados_tipologia.columns = ['Tipologia', 'Poblacion Total 2010', 'Poblacion Total 2020', 'Viviendas Total 2010',
                                'Viviendas Total 2020', 'Manzanas', 'Area (ha)','% Poblacion Total 2010', '% Poblacion Total 2020', '% Viviendas Total 2010',
                                '% Viviendas Total 2020', '% Manzanas', '% Area (ha)',
                                'Diferencia Poblacion', 'Diferencia Viviendas',
                                'Tasa Crecimiento Poblacion (%)', 'Tasa Crecimiento Viviendas (%)',
                                'Promedio de tamaño de Manzana', 'Densidad poblacional 2010', 'Densidad poblacional 2020',
                                'Densidad habitacional 2010', 'Densidad habitacional 2020','Habitantes por vivienda 2010', 'Habitantes por vivienda 2020'
                                ]

# Limitar la cantidad de decimales en el DataFrame
resultados_tipologia = resultados_tipologia.round({
    'Poblacion Total 2010': 0,
    'Poblacion Total 2020': 0,
    'Viviendas Total 2010': 0,
    'Viviendas Total 2020': 0,
    'Area (ha)': 3,
    '% Poblacion Total 2010': 3,
    '% Poblacion Total 2020': 3,
    '% Viviendas Total 2010': 3,
    '% Viviendas Total 2020': 3,
    '% Area (ha)': 3,
    '% Manzanas' : 3,
    'Diferencia Poblacion': 0,
    'Diferencia Viviendas': 0,
    'Tasa Crecimiento Poblacion (%)': 3,
    'Tasa Crecimiento Viviendas (%)': 3,
    'Promedio de tamaño de Manzana': 3,
    'Densidad poblacional 2010': 3,
    'Densidad poblacional 2020': 3,
    'Densidad habitacional 2010': 3,
    'Densidad habitacional 2020': 3,
    'Habitantes por vivienda 2010': 3,
    'Habitantes por vivienda 2020': 3
})

# Guardar los resultados del conjunto de datos completo en una hoja Excel
with pd.ExcelWriter(r'C:\Users\ADMIN\Desktop\Ejercicio\Yucatán\resultados.xlsx', engine='xlsxwriter') as writer:
    resultados_tipologia.to_excel(writer, sheet_name='Zona Metropolitana', index=False)

    # Iterar sobre los municipios y realizar el mismo procedimiento
    for municipio in MAPA_TIPO_gdf['CVE_MUN'].unique():
        resultados_municipio = MAPA_TIPO_gdf[MAPA_TIPO_gdf['CVE_MUN'] == municipio].groupby('TIPOLOGIA').agg({
            'POPTOT2010': 'sum',
            'POPTOT2020': 'sum',
            'VIVTOT2010': 'sum',
            'VIVTOT2020': 'sum',
            'geometry': 'count',
            'area_ha': 'sum'
        }).reset_index()

        # Calcular el total
        total_municipio = resultados_municipio.sum(numeric_only=True)

        # Agregar el total a los resultados
        resultados_municipio.loc['Total'] = total_municipio

        # Calcular los porcentajes de cada tipología en el total
        resultados_municipio['% Población Total 2010'] = (resultados_municipio['POPTOT2010'] / resultados_municipio.loc['Total', 'POPTOT2010']) * 100
        resultados_municipio['% Población Total 2020'] = (resultados_municipio['POPTOT2020'] / resultados_municipio.loc['Total', 'POPTOT2020']) * 100
        resultados_municipio['% Viviendas Total 2010'] = (resultados_municipio['VIVTOT2010'] / resultados_municipio.loc['Total', 'VIVTOT2010']) * 100
        resultados_municipio['% Viviendas Total 2020'] = (resultados_municipio['VIVTOT2020'] / resultados_municipio.loc['Total', 'VIVTOT2020']) * 100
        resultados_municipio['% Manzanas'] = (resultados_municipio['geometry'] / resultados_municipio.loc['Total', 'geometry']) * 100
        resultados_municipio['% Area (ha)'] = (resultados_municipio['area_ha'] / resultados_municipio.loc['Total', 'area_ha']) * 100

        # Calcular la diferencia de población y vivienda entre 2020 y 2010
        resultados_municipio['Diferencia Población'] = resultados_municipio['POPTOT2020'] - resultados_municipio['POPTOT2010']
        resultados_municipio['Diferencia Viviendas'] = resultados_municipio['VIVTOT2020'] - resultados_municipio['VIVTOT2010']

        # Calcular la tasa de crecimiento por tipología y total
        resultados_municipio['Tasa Crecimiento Población (%)'] = (((resultados_municipio['POPTOT2020'] / resultados_municipio['POPTOT2010']) ** (1/10)) - 1) * 100
        resultados_municipio['Tasa Crecimiento Viviendas (%)'] = (((resultados_municipio['VIVTOT2020'] / resultados_municipio['VIVTOT2010']) ** (1/10)) - 1) * 100

        # Calcular la cantidad de geometrías sobre área, población sobre área, y población sobre vivienda de 2010 y 2020
        resultados_municipio['Promedio de tamaño de Manzana'] = resultados_municipio['geometry'] / resultados_municipio['area_ha']
        resultados_municipio['Densidad poblacional 2010'] = resultados_municipio['POPTOT2010'] / resultados_municipio['area_ha']
        resultados_municipio['Densidad poblacional 2020'] = resultados_municipio['POPTOT2020'] / resultados_municipio['area_ha']
        resultados_municipio['Densidad habitacional 2010'] = resultados_municipio['VIVTOT2010'] / resultados_municipio['area_ha']
        resultados_municipio['Densidad habitacional 2020'] = resultados_municipio['VIVTOT2020'] / resultados_municipio['area_ha']
        resultados_municipio['Habitantes por vivienda 2010'] = resultados_municipio['POPTOT2010'] / resultados_municipio['VIVTOT2010']
        resultados_municipio['Habitantes por vivienda 2020'] = resultados_municipio['POPTOT2020'] / resultados_municipio['VIVTOT2020']

        # Renombrar las columnas
        resultados_municipio.columns = ['Tipologia', 'Poblacion Total 2010', 'Poblacion Total 2020', 'Viviendas Total 2010',
                                        'Viviendas Total 2020', 'Manzanas', 'Area (ha)','% Poblacion Total 2010', '% Poblacion Total 2020', '% Viviendas Total 2010',
                                        '% Viviendas Total 2020', '% Manzanas', '% Area (ha)',
                                        'Diferencia Poblacion', 'Diferencia Viviendas',
                                        'Tasa Crecimiento Poblacion (%)', 'Tasa Crecimiento Viviendas (%)',
                                        'Promedio de tamaño de Manzana', 'Densidad poblacional 2010', 'Densidad poblacional 2020',
                                        'Densidad habitacional 2010', 'Densidad habitacional 2020','Habitantes por vivienda 2010', 'Habitantes por vivienda 2020'
                                        ]

        # Limitar la cantidad de decimales en el DataFrame
        resultados_municipio = resultados_municipio.round({
            'Poblacion Total 2010': 0,
            'Poblacion Total 2020': 0,
            'Viviendas Total 2010': 0,
            'Viviendas Total 2020': 0,
            'Area (ha)': 3,
            '% Poblacion Total 2010': 3,
            '% Poblacion Total 2020': 3,
            '% Viviendas Total 2010': 3,
            '% Viviendas Total 2020': 3,
            '% Area (ha)': 3,
            '% Manzanas' : 3,
            'Diferencia Poblacion': 0,
            'Diferencia Viviendas': 0,
            'Tasa Crecimiento Poblacion (%)': 3,
            'Tasa Crecimiento Viviendas (%)': 3,
            'Promedio de tamaño de Manzana': 3,
            'Densidad poblacional 2010': 3,
            'Densidad poblacional 2020': 3,
            'Densidad habitacional 2010': 3,
            'Densidad habitacional 2020': 3,
            'Habitantes por vivienda 2010': 3,
            'Habitantes por vivienda 2020': 3
        })

        # Guardar los resultados en una nueva hoja Excel
        resultados_municipio.to_excel(writer, sheet_name=f'Municipio_{municipio}', index=False)
        
# Ajustar el ancho de las columnas al contenido en todas las hojas
    for sheet in writer.sheets.values():
        for idx, col in enumerate(resultados_tipologia.columns):
            max_len = max(resultados_tipologia[col].astype(str).map(len).max(), len(col))
            sheet.set_column(idx, idx, max_len)
            
# Mostrar los resultados del conjunto completo
print(resultados_tipologia)